# Slicing chunks and beyond

The newest and coolest way to store data in python-blosc2 is through a SChunk (super-chunk) object. Here the data is split into chunks of the same size. So in the past, the only way of working with it was chunk by chunk (see  tutorials-basics.ipynb). But now, python-blosc2 can retrieve, update or append data all at once (i.e. avoiding doing it chunk by chunk). To see how this works, let's first create our SChunk.

In [1]:
import blosc2
import numpy as np

nchunks = 10
data = np.arange(200 * 1000 * nchunks, dtype=np.int32)
cparams = {"typesize": 4}
schunk = blosc2.SChunk(chunksize=200 * 1000 * 4, data=data, cparams=cparams)

It is important to set the `typesize` correctly as these methods will work with items and not with bytes.

## Getting data in a SChunk

Let's begin by retrieving the data from the whole SChunk. We could use the `decompress_chunk` method:

In [2]:
out = np.empty(200 * 1000 * nchunks, dtype=np.int32)
for i in range(nchunks):
    schunk.decompress_chunk(i, out[200 * 1000 * i : 200 * 1000 * (i + 1)])

But instead of the code above, we can simply use the `__getitem__` or the `get_slice` methods. Let's begin with `__getitem__`:

In [3]:
out_slice = schunk[:]
print(out_slice[:4])

b'\x00\x00\x00\x00'


As you can see, the data is returned as a bytestring. If we want to better visualize the data, we will use `get_slice`. You can pass any Python object (supporting the Buffer Protocol) as the `out` param to fill it with the data.

In [4]:
out_slice = np.empty(200 * 1000 * nchunks, dtype=np.int32)
schunk.get_slice(out=out_slice)
np.array_equal(out, out_slice)
print(out_slice[:4])

[0 1 2 3]


That's much better!

## Setting data in a SChunk

We can also set the data of an area to any python object supporting the Buffer Protocol. Let's see a quick example:

In [5]:
start = 34
stop = 1000 * 200 * 4
new_value = np.ones(stop - start, dtype=np.int32)
schunk[start:stop] = new_value

So now, we are able to get or set data all at once. But what if we would like to add data? Well, you can still do it with `__setitem__`. Indeed, this method can update and append data at the same time. To do so, `stop` will be the new SChunk nitems:

In [6]:
schunk_nelems = 1000 * 200 * nchunks

new_value = np.zeros(1000 * 200 * 2 + 53, dtype=np.int32)
start = schunk_nelems - 123
new_nitems = start + new_value.size
schunk[start:new_nitems] = new_value

## Getting a SChunk from/as a contiguous buffer

Furthermore, you can pass from a SChunk to a contiguous buffer and vice versa. Let's get that buffer:

In [7]:
buf = schunk.to_cframe()
buf[:5]

b'\x9e\xa8b2f'

And now the other way around:

In [8]:
schunk2 = blosc2.schunk_from_cframe(cframe=buf, copy=True)

In this case we set the `copy` param to `True`. If you do not want to copy the buffer,
be mindful that you will have to keep its reference until you do not
want the SChunk anymore.

## Compressing NumPy arrays

If the object you want to get as a compressed buffer is a NumPy array, you can use the newer and faster functions to store it in-memory or on-disk.

### In-memory

To store it in-memory you can use `pack_array2`. In comparison with its former version, it is faster (see `pack_compress.py` bench)  and does not have the 2 GB size limitation.

In [9]:
np_array = np.arange(2**30, dtype=np.int32)

packed_arr2 = blosc2.pack_array2(np_array)
unpacked_arr2 = blosc2.unpack_array2(packed_arr2)

### On-disk

To perform the same but store the buffer on-disk you would use `save_array` and `load_array` like so:

In [10]:
blosc2.save_array(np_array, urlpath="ondisk_array.b2frame", mode="w")
np_array2 = blosc2.load_array("ondisk_array.b2frame")
np.array_equal(np_array, np_array2)

# Remove it
blosc2.remove_urlpath("ondisk_array.b2frame")

# Conclusions

Now python-blosc2 has an easy way of creating, getting, setting, deleting and expanding data in a SChunk. Moreover, you can get a contiguous compressed representation (aka [cframe](https://github.com/Blosc/c-blosc2/blob/main/README_CFRAME_FORMAT.rst)) of it and create it again latter. And you can do the same with NumPy arrays faster than with the former functions.
